In [2]:
import os
import pdb

import tqdm
import random
import json


In [35]:
split_files = {
            'train': 'egotracks_train.json',
            'val': 'egotracks_val.json',            # there is no test
            'test': 'egotracks_challenge_test_unannotated.json'
        }
meta_dir='/vision/hwjiang/episodic-memory/egotrack/data'
data_dir='/vision/hwjiang/episodic-memory/VQ2D/data'
split = 'train'

In [36]:
anno_processed_path = os.path.join('./data', '{}_egotracks_anno.json'.format(split))
target_split_fp = split_files[split]
ann_file = os.path.join(meta_dir, target_split_fp)
with open(ann_file) as f:
    anno_json = json.load(f)

In [28]:
def get_query_path(sample):
    clip_name = sample['clip_uid']
    image_name = int(sample["visual_crop"]["frame_number"])# "{}/frame_{:07d}.jpg"
    #print(clip_name)
    image_path = os.path.join(data_dir, 'images_hw', str(clip_name))
    image_path = os.path.join(image_path, "frame_{:07d}.jpg".format(image_name + 1))
    return image_path

In [37]:
annotations, n_samples, n_samples_valid, n_no_lt_track = [], 0, 0, 0
for video_data in anno_json['videos']:
    for clip_data in video_data['clips']:
        for clip_anno in clip_data['annotations']:
            for qset_id, qset in clip_anno['query_sets'].items():
                if not qset['is_valid']:
                    continue
                response_track_frame_ids = []
                for frame_it in qset['response_track']:
                    response_track_frame_ids.append(int(frame_it['frame_number']))
                frame_id_min, frame_id_max = min(response_track_frame_ids), max(response_track_frame_ids)
                lt_track_frame_ids = []
                # print(qset['lt_track'])

                if 'lt_track' in qset.keys():
                    lt_track = qset['lt_track']
                else:
                    lt_track = qset['response_track']
                    n_no_lt_track += 1
                for frame_it in lt_track:
                    lt_track_frame_ids.append(int(frame_it['frame_number']))
                curr_anno = {
                    "metadata": {
                        "video_uid": video_data["video_uid"],
                        "video_start_sec": clip_data["video_start_sec"],
                        "video_end_sec": clip_data["video_end_sec"],
                        "clip_fps": clip_data["clip_fps"],
                    },
                    "clip_uid": clip_data["clip_uid"],
                    "clip_fps": clip_data["clip_fps"],
                    "query_set": qset_id,
                    "query_frame": qset["query_frame"],
                    "response_track": sorted(qset["response_track"], key=lambda x: x['frame_number']),
                    "response_track_valid_range": [frame_id_min, frame_id_max], 
                    "lt_track": sorted(lt_track, key=lambda x: x['frame_number']),
                    "lt_track_frame_ids": sorted(lt_track_frame_ids),
                    "visual_crop": qset["visual_crop"],
                    "object_title": qset["object_title"],
                    # Assign a unique ID to this annotation for the dataset
                    "dataset_uid": f"{split}_{n_samples_valid:010d}"
                }
                query_path = get_query_path(curr_anno)
                if os.path.isfile(query_path):
                    annotations.append(curr_anno)
                    n_samples_valid += 1
                elif split == 'train' and clip_data["clip_uid"] == '859ed253-d752-4f1b-adc3-c76599117d6e':
                    print(query_path, curr_anno['clip_uid'], curr_anno['visual_crop']['frame_number'], curr_anno['visual_crop'])
                n_samples += 1
print('Find {} data samples, {} valid (query path exist), {} no lt_track'.format(n_samples, n_samples_valid, n_no_lt_track))
with open(anno_processed_path, 'w') as ff:
    json.dump(annotations, ff)

Find 13637 data samples, 13616 valid (query path exist), 497 no lt_track
